In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

__Напишем функцию, которая поможет улучшить текст для обучения модели__

In [2]:
def dataset_improvement(dataset):
    
    """This feature improves the dataset.
    Makes all letters in words lowercase and removes non-alphabetic characters.
    Attention: the function uses regular expressions."""
    
    dataset['title'] = dataset['title'].str.lower().str.replace(r'\W',r' ')
    return dataset

## 1 этап. Подбор оптимальной модели и параметров векторизации.

__Прочитаем данные из файла обучения "train.tsv" для подбора наилучшей модели обучения, ее параметров и тестирования__ 

In [3]:
train_data = pd.read_csv('dataset/train.tsv', sep='\t')
train_data

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0
5754,Правительство застроит Россию нефтепродуктопро...,0
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1
5756,Туркмения в декабре начнет поставки газа в Китай,0


__Применим функцию улучшения текста к тренировочным данным__

In [4]:
train_data = dataset_improvement(train_data)

C:\Users\anton.babushkin\AppData\Local\Temp\ipykernel_8516\2645509092.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['title'] = dataset['title'].str.lower().str.replace(r'\W',r' ')


__Создадим множество новостей и множество ответов о лживости новости__

In [5]:
X = train_data['title']
y = train_data['is_fake']

__Для тестирования подходящей модели и подбора оптимальных параметров разобьем тренировочные данные на данные для обучения и тестирования__

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

__Для хорошей работы модели необходимо каждое слово в тексте векторизовать. Воспользуемся встроенной функцией SKlearn CountVectorizer и применим ее к нашим множествам. При тестировании модели были найдены оптимальные параметры векторизации: ngram_range=(1, 2), остальные параметры оставляем дефолтными__

In [7]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer.fit(X_train)
vectorizer.fit(X_test)
X_train_vect = vectorizer.transform(X_train)
X_test_vect = vectorizer.transform(X_test)

__В ходе исследования была подобрана модель LogisticRegression c дефолтными параметрами. Она показала 83%-85% совпадение с ответами тестовой выборкой__

In [8]:
log_reg = LogisticRegression()
log_reg.fit(X_train_vect, y_train)

LogisticRegression()

In [9]:
log_reg.score(X_test_vect, y_test)

0.8385416666666666


## 2 этап. Обучение модели на всех тренировочных данных, решение тестовой задачи


__Обучим модель на всех тренировочных данных__

In [10]:
vectorizer.fit(X)
X_train_vect = vectorizer.transform(X)
log_reg.fit(X_train_vect, y)

LogisticRegression()

__Загрузим тестовый файл, применим функцию улучшения текста, векторизуем и применим модель для получения значений поля is_fake__

In [11]:
test_data = pd.read_csv('dataset/test.tsv', sep='\t')
test_data_improvement = dataset_improvement(test_data.copy())
X = test_data_improvement['title']
test_data['is_fake'] = log_reg.predict(vectorizer.transform(X))
test_data

C:\Users\anton.babushkin\AppData\Local\Temp\ipykernel_8516\2645509092.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['title'] = dataset['title'].str.lower().str.replace(r'\W',r' ')


,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
...,...,...
995,Прокуратура заподозрила Явлинского в авторитар...,0
996,В День Победы стратегические ракетоносцы Ту-16...,1
997,СК возбудил дело против авиакомпании «Победа» ...,1
998,Криптомонетный двор Туркменистана выпустил юби...,0


__Сохраним данные в файл__

In [12]:
test_data.to_csv('prediction.tsv', sep='\t', index=False)